In [3]:
%load_ext autoreload
%autoreload 2


In [4]:
from transformers import AutoTokenizer, AutoModelForMaskedLM
from transformers import DataCollatorForLanguageModeling
from transformers import Trainer, TrainingArguments
import pickle
from mlm_dataset import MLMDataset


In [5]:
def init_model(model_name):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    tokenizer.eos_token = tokenizer.sep_token
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForMaskedLM.from_pretrained(model_name)
    return tokenizer, model


def load_data(path):
    with open(path, 'rb') as f:
        data = pickle.load(f)
    return list(data)


def get_train_test_data(data, split=0.8):
    num_examples = len(data)
    train_data = data[:int(num_examples * split)]
    test_data = data[int(num_examples * split):]
    return train_data, test_data


def get_dataset(train_data, test_data, tokenizer):
    train_dataset = MLMDataset(train_data, tokenizer)
    test_dataset = MLMDataset(test_data, tokenizer)
    return {'train': train_dataset, 'test': test_dataset}




In [12]:
print("Loading Model")
tokenizer, model = init_model("bert-base-uncased")

print("Loading Data")
data = load_data("../kg_paths.pkl")


Loading Model


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [13]:
print("Creating Dataset")
train_data, test_data = get_train_test_data(data)
lm_dataset = get_dataset(train_data, test_data, tokenizer)


Creating Dataset


In [14]:
print(next(iter(lm_dataset["train"]))['input_ids'].reshape(1, -1).shape)

torch.Size([1, 512])


In [15]:
tokenizer.save_pretrained("custom_bert_mlm")

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15, mlm=True, return_tensors='pt')


In [16]:
training_args = TrainingArguments(
    output_dir="custom_bert_mlm",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=3,
    weight_decay=0.01,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    disable_tqdm=False
)

In [17]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset["train"],
    eval_dataset=lm_dataset["test"],
    data_collator=data_collator,
)

trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:9                                                                                    │
│                                                                                                  │
│    6 │   data_collator=data_collator,                                                            │
│    7 )                                                                                           │
│    8                                                                                             │
│ ❱  9 trainer.train()                                                                             │
│   10                                                                                             │
│                                                                                                  │
│ C:\Users\mihir\PycharmProjects\assignment-2-env\lib\site-packages\transformers\trainer.py:1649   │
│ in train                                                                                         │
│                                                                                                  │
│   1646 │   │   │   args=args,                                                                    │
│   1647 │   │   │   resume_from_checkpoint=resume_from_checkpoint,                                │
│   1648 │   │   │   trial=trial,                                                                  │
│ ❱ 1649 │   │   │   ignore_keys_for_eval=ignore_keys_for_eval,                                    │
│   1650 │   │   )                                                                                 │
│   1651 │                                                                                         │
│   1652 │   def _inner_training_loop(                                                             │
│                                                                                                  │
│ C:\Users\mihir\PycharmProjects\assignment-2-env\lib\site-packages\transformers\trainer.py:1938   │
│ in _inner_training_loop                                                                          │
│                                                                                                  │
│   1935 │   │   │   │   │   self.control = self.callback_handler.on_step_begin(args, self.state,  │
│   1936 │   │   │   │                                                                             │
│   1937 │   │   │   │   with self.accelerator.accumulate(model):                                  │
│ ❱ 1938 │   │   │   │   │   tr_loss_step = self.training_step(model, inputs)                      │
│   1939 │   │   │   │                                                                             │
│   1940 │   │   │   │   if (                                                                      │
│   1941 │   │   │   │   │   args.logging_nan_inf_filter                                           │
│                                                                                                  │
│ C:\Users\mihir\PycharmProjects\assignment-2-env\lib\site-packages\transformers\trainer.py:2770   │
│ in training_step                                                                                 │
│                                                                                                  │
│   2767 │   │   │   with amp.scale_loss(loss, self.optimizer) as scaled_loss:                     │
│   2768 │   │   │   │   scaled_loss.backward()                                                    │
│   2769 │   │   else:                                                                             │
│ ❱ 2770 │   │   │   self.accelerator.backward(loss)                                               │
│   2771 │   │                                                                                     │
│   2772 │   │   return loss.detach() / self.args.gradient_ac